# Checking enviroment states and action sets

In [1]:
import sys
import os.path
sys.path.append("..")
from SAC_Agent import *
from ENV_DETAILS import *

2023-11-20 08:46:09.441078: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-20 08:46:09.504749: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-20 08:46:09.520649: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-20 08:46:09.827621: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: li

Using TensorFlow backend
Num devices available:  [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


2023-11-20 08:46:10.672836: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-11-20 08:46:10.718543: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-11-20 08:46:10.718724: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


In [2]:
ENV = "MountainCarContinuous-v0"
SUCESS_CRITERIA_VALUE = ENV_DETAILS[ENV]["SUCESS_CRITERIA_VALUE"]
SUCESS_CRITERIA_EPOCH = ENV_DETAILS[ENV]["SUCESS_CRITERIA_EPOCH"]
EPISODES = ENV_DETAILS[ENV]["EPISODES"]

In [3]:

env = gym.make(ENV)
env


<TimeLimit<OrderEnforcing<Continuous_MountainCarEnv<MountainCarContinuous-v0>>>>

In [4]:
s = env.observation_space.sample()
s

array([-0.6837731 ,  0.00882459], dtype=float32)

In [5]:
env.observation_space, env.observation_space.shape

(Box([-1.2  -0.07], [0.6  0.07], (2,), float32), (2,))

In [6]:
s = env.reset()
s[0]

-0.46692708

In [8]:
s = env.reset()
env.action_space, env.action_space, env.step(s)

(Box(-1.0, 1.0, (1,), float32),
 Box(-1.0, 1.0, (1,), float32),
 (array([-0.45186692, -0.00121874], dtype=float32),
  -0.02030837898803526,
  False,
  {}))

### Training/Hyperparam run

In [9]:
%tensorboard --logdir logs_general/hyper

UsageError: Line magic function `%tensorboard` not found.


In [10]:
TUNING_TYPE = "BAYES"
HYPERPARAM_TUNING = True
RUN_TRAINING = True
writer= "Training/fit_A3C/"

In [11]:
if HYPERPARAM_TUNING:

    dir = r"Hyperparam_kt_sac"
    project_name = "keras_tunning_soft"

    tuner = kt.BayesianOptimization(
            MyHyperModel( hyper_dir = dir+"/"+project_name,  writer = "logs_hyper/sac/",
                        end_of_episode = EPISODES, evaluation_epoch = env._max_episode_steps, training_steps = 600000,
                sucess_criteria_epochs = SUCESS_CRITERIA_EPOCH, sucess_criteria_value= SUCESS_CRITERIA_VALUE,
                discount_min = 0.98, discount_max = 0.99,
                lr_actor_min = 0.000001, lr_actor_max = 0.001,
                lr_critic_1_min = 0.000001, lr_critic_1_max = 0.001,
                dense_min = 32, dense_max = 256,
                environment_name=ENV,
                tau_min = 0.001, tau_max = 0.1,
                reward_scaler = 1, num_layers_act = 3, num_layers_crit =3,
                train_epochs = 50),
            objective= kt.Objective('total_train_reward', direction="max"), 
            max_trials = 30,
            directory=dir,
            project_name=project_name
        )
    tuner.search(x=[0], y=[1])
else : 
        
        writer= "Training/fit_PPO/"

        training_steps = 1000000
        lr_actor= 0.0001
        lr_critic_1= 0.0001
        lr_alpha= 0.0001
        entropy_factor = 0.1
        discount = 0.99
        dense_layers_actor = [128,64]
        num_layers_actor = 2
        dense_units_critic = [128,64]
        num_layers_crit = 2
        tau = 0.01
        save_factor=50000
        reward_scaler = 1
        model_path = './checkpoints/SACagent'

        if RUN_TRAINING:

            print("Acquiring parameters ....")

            model = run_training(training_steps = training_steps,
                                discount = discount,  
                                dense_units_act = dense_layers_actor,  
                                dense_units_crit = dense_units_critic, 
                                num_layer_a = num_layers_actor, 
                                num_layer_c = num_layers_crit, 
                                writer = writer, 
                                end_of_episode = EPISODES, 
                                save_factor = save_factor, 
                                sucess_criteria_epochs = SUCESS_CRITERIA_EPOCH , 
                                sucess_criteria_value = SUCESS_CRITERIA_VALUE, 
                                environment_name = ENV, 
                                reward_scaler = 1 , 
                                evaluation_epoch = env._max_episode_steps, 
                                return_agent = True,
                                lr_actor = lr_actor, 
                                lr_critic_1 = lr_critic_1, 
                                lr_alpha = lr_alpha,
                                tau = tau,
                                train_epochs = 50,
                                model_path = model_path
                                )
        else:
        
            model = SAC(
                discount = discount, 
                dense_units_act = dense_layers_actor,
                dense_units_crit= dense_units_critic, 
                num_layer_act  = num_layers_actor, 
                num_layer_crit= num_layers_crit,
                writer = writer,
                trial_n = get_valid_trials_number(writer),
                end_of_episode = EPISODES,
                evaluation_epoch = env._max_episode_steps,
                environment_name = ENV,
                reward_scaler = reward_scaler,
                lr_actor = lr_actor, lr_critic = lr_critic_1, lr_alpha= lr_alpha, 
                tau= tau,
                train_epochs = 50
            )
            model.load_weights(model_path)

            

Reloading Tuner from Hyperparam_kt_sac/keras_tunning_soft/tuner0.json


RuntimeError: Error reloading `Oracle` from existing project. If you did not mean to reload from an existing project, change the `project_name` or pass `overwrite=True` when creating the `Tuner`. Found existing project at: Hyperparam_kt_sac/keras_tunning_soft

In [ ]:
exploration_tech = "soft_sac"
val_env = gym.make(ENV, render_mode = "rgb_array")
dir = r"Hyperparam_kt_sac"

model = tuner.get_best_models()[0]
final_rewards = final_evaluation(model,val_env,n_tries=200, exploration=exploration_tech,  video_name = "./sac_"+exploration_tech+"_video.mp4")
print("Final mean reward '",exploration_tech,"':", np.mean(final_rewards))